In [2]:
!pip3 install imblearn
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import re
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV,StratifiedKFold,cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.dummy import DummyClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix,recall_score,roc_auc_score,roc_curve,auc
%matplotlib inline

from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek

     |████████████████████████████████| 199 kB 2.8 MB/s eta 0:00:01


In [4]:
train= pd.read_csv('Vehicle Loan Default Prediction- L&T Data Science Finhack /train.csv',infer_datetime_format=True)
test= pd.read_csv('Vehicle Loan Default Prediction- L&T Data Science Finhack /test_bqCt9Pv.csv',infer_datetime_format=True)

In [5]:
train.shape

(210485, 41)

In [6]:
test.shape

(112392, 40)

In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210485 entries, 0 to 210484
Data columns (total 41 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   UniqueID                             210485 non-null  int64  
 1   disbursed_amount                     210484 non-null  float64
 2   asset_cost                           210484 non-null  float64
 3   ltv                                  210484 non-null  float64
 4   branch_id                            210484 non-null  float64
 5   supplier_id                          210484 non-null  float64
 6   manufacturer_id                      210484 non-null  float64
 7   Current_pincode_ID                   210484 non-null  float64
 8   Date.of.Birth                        210484 non-null  object 
 9   Employment.Type                      203420 non-null  object 
 10  DisbursalDate                        210484 non-null  object 
 11  State_ID     

In [10]:
train.head()

,UniqueID,disbursed_amount,asset_cost,ltv,branch_id,supplier_id,manufacturer_id,Current_pincode_ID,Date.of.Birth,Employment.Type,...,SEC.SANCTIONED.AMOUNT,SEC.DISBURSED.AMOUNT,PRIMARY.INSTAL.AMT,SEC.INSTAL.AMT,NEW.ACCTS.IN.LAST.SIX.MONTHS,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,AVERAGE.ACCT.AGE,CREDIT.HISTORY.LENGTH,NO.OF_INQUIRIES,loan_default
0,420825,50578.0,58400.0,89.55,67.0,22807.0,45.0,1441.0,01-01-84,Salaried,...,0.0,0.0,0.0,0.0,0.0,0.0,0yrs 0mon,0yrs 0mon,0.0,0.0
1,537409,47145.0,65550.0,73.23,67.0,22807.0,45.0,1502.0,31-07-85,Self employed,...,0.0,0.0,1991.0,0.0,0.0,1.0,1yrs 11mon,1yrs 11mon,0.0,1.0
2,417566,53278.0,61360.0,89.63,67.0,22807.0,45.0,1497.0,24-08-85,Self employed,...,0.0,0.0,0.0,0.0,0.0,0.0,0yrs 0mon,0yrs 0mon,0.0,0.0
3,624493,57513.0,66113.0,88.48,67.0,22807.0,45.0,1501.0,30-12-93,Self employed,...,0.0,0.0,31.0,0.0,0.0,0.0,0yrs 8mon,1yrs 3mon,1.0,1.0
4,539055,52378.0,60300.0,88.39,67.0,22807.0,45.0,1495.0,09-12-77,Self employed,...,0.0,0.0,0.0,0.0,0.0,0.0,0yrs 0mon,0yrs 0mon,1.0,1.0


In [11]:
train.nunique()

UniqueID                               210485
disbursed_amount                        23191
asset_cost                              43785
ltv                                      6504
branch_id                                  82
supplier_id                              2782
manufacturer_id                            11
Current_pincode_ID                       6346
Date.of.Birth                           15280
Employment.Type                             2
DisbursalDate                              84
State_ID                                   22
Employee_code_ID                         2470
MobileNo_Avl_Flag                           1
Aadhar_flag                                 2
PAN_flag                                    2
VoterID_flag                                2
Driving_flag                                2
Passport_flag                               2
PERFORM_CNS.SCORE                         573
PERFORM_CNS.SCORE.DESCRIPTION              20
PRI.NO.OF.ACCTS                   

In [13]:
train.loan_default.value_counts()

0.0    165109
1.0     45375
Name: loan_default, dtype: int64

In [15]:
train.isna().sum()

UniqueID                                  0
disbursed_amount                          1
asset_cost                                1
ltv                                       1
branch_id                                 1
supplier_id                               1
manufacturer_id                           1
Current_pincode_ID                        1
Date.of.Birth                             1
Employment.Type                        7065
DisbursalDate                             1
State_ID                                  1
Employee_code_ID                          1
MobileNo_Avl_Flag                         1
Aadhar_flag                               1
PAN_flag                                  1
VoterID_flag                              1
Driving_flag                              1
Passport_flag                             1
PERFORM_CNS.SCORE                         1
PERFORM_CNS.SCORE.DESCRIPTION             1
PRI.NO.OF.ACCTS                           1
PRI.ACTIVE.ACCTS                

In [17]:
train['Employment.Type'].value_counts()

Self employed    114889
Salaried          88531
Name: Employment.Type, dtype: int64

In [19]:
train.isna().sum()

UniqueID                                  0
disbursed_amount                          1
asset_cost                                1
ltv                                       1
branch_id                                 1
supplier_id                               1
manufacturer_id                           1
Current_pincode_ID                        1
Date.of.Birth                             1
Employment.Type                        7065
DisbursalDate                             1
State_ID                                  1
Employee_code_ID                          1
MobileNo_Avl_Flag                         1
Aadhar_flag                               1
PAN_flag                                  1
VoterID_flag                              1
Driving_flag                              1
Passport_flag                             1
PERFORM_CNS.SCORE                         1
PERFORM_CNS.SCORE.DESCRIPTION             1
PRI.NO.OF.ACCTS                           1
PRI.ACTIVE.ACCTS                

In [20]:
train.describe()

,UniqueID,disbursed_amount,asset_cost,ltv,branch_id,supplier_id,manufacturer_id,Current_pincode_ID,State_ID,Employee_code_ID,...,SEC.OVERDUE.ACCTS,SEC.CURRENT.BALANCE,SEC.SANCTIONED.AMOUNT,SEC.DISBURSED.AMOUNT,PRIMARY.INSTAL.AMT,SEC.INSTAL.AMT,NEW.ACCTS.IN.LAST.SIX.MONTHS,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,NO.OF_INQUIRIES,loan_default
count,210485.000000,210484.000000,2.104840e+05,210484.000000,210484.000000,210484.000000,210484.000000,210484.000000,210484.000000,210484.000000,...,210484.000000,2.104840e+05,2.104840e+05,2.104840e+05,2.104840e+05,2.104840e+05,210484.000000,210484.000000,210484.000000,210484.000000
mean,533369.007074,54387.142719,7.593347e+04,74.726106,71.503535,19580.235457,69.268434,3449.457959,7.148957,1383.904059,...,0.007440,5.527539e+03,7.335454e+03,7.219209e+03,1.288114e+04,3.131016e+02,0.379473,0.097167,0.204747,0.215575
std,68358.531124,13019.372076,1.902728e+04,11.488276,69.607774,3472.639798,22.060119,2241.671656,4.418607,855.970853,...,0.112213,1.761204e+05,1.867063e+05,1.861247e+05,1.552513e+05,1.567723e+04,0.951490,0.384135,0.702816,0.411221
min,645.000000,13320.000000,3.712900e+04,10.030000,1.000000,10524.000000,45.000000,1.000000,1.000000,3.000000,...,0.000000,-5.746470e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000
25%,473903.000000,47149.000000,6.574300e+04,68.820000,13.000000,16445.000000,48.000000,1515.000000,4.000000,655.000000,...,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000
50%,532134.000000,53803.000000,7.096600e+04,76.770000,61.000000,20292.000000,86.000000,2989.000000,6.000000,1318.000000,...,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000
75%,592343.000000,60443.000000,7.932525e+04,83.700000,121.000000,22976.000000,86.000000,5706.000000,10.000000,2110.000000,...,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,1.984000e+03,0.000000e+00,0.000000,0.000000,0.000000,0.000000
max,658676.000000,990572.000000,1.628992e+06,95.000000,261.000000,24803.000000,156.000000,7345.000000,22.000000,3792.000000,...,8.000000,3.603285e+07,3.000000e+07,3.000000e+07,2.564281e+07,4.170901e+06,35.000000,20.000000,36.000000,1.000000
